In [1]:
import gensim
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
#nltk.download()
import string 
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import pickle 
import pandas as pd 
import os

In [2]:



#reading documents

data=[]
docLabels=[]
tot=0
dub=0
for i in range(2011,2014):
    pathname = os.path.join("Filtered", str(i))
    files=os.listdir(pathname)
    print(pathname)
    print(len(files))
    tot=tot+len(files)
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        if caseid not in docLabels:
            
            try:
                #print(pathname+"/"+file)
                data.append(open(pathname+"/"+file).read())
                docLabels.append(caseid)
            except:
                print("error")
                print(file)
                print(i)
                continue
            


        else:
            dub=dub+1
            
            
print(len(docLabels))
print(len(data))
print(tot)
print(dub)

Filtered/2011
568
Filtered/2012
618
Filtered/2013
485
1663
1663
1671
8


In [3]:


#case metadata
casedata = pd.read_pickle('casedata.pkl')







In [4]:



def clean(text):

    text=text.lower()
    table = str.maketrans("", "", string.punctuation)
    wnl = nltk.WordNetLemmatizer()
    
    
    text=text.translate(table)
    #print("without punctuation ")
    #print(text)
    token = nltk.word_tokenize(text)
    #print("tokenized")
    #print(token)
    

    stop_words = set(stopwords.words('english'))
    filtered_sentence = []

    for w in token:
        if w not in stop_words:
            filtered_sentence.append(w)
   
    token=filtered_sentence
    #print("without stop words")
    #print(token)
    token=[wnl.lemmatize(t) for t in token]
    #print("after lemma")
    #print(token)
    return token

new_data=[]
for d in data:
    d=clean(d)
    new_data.append(d)
data=new_data

In [5]:
#create iterator of data and doclabels
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])


In [6]:

it = LabeledLineSentence(data, docLabels)


In [7]:
model = gensim.models.Doc2Vec(size=25, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)

#training 
for epoch in range(100):
    #print('iteration '+str(epoch+1))
    model.train(it,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

model.save('doc2vec.model')
print("model saved")

/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  


model saved


In [8]:
d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model')

In [13]:
pd.set_option('display.max_rows', len(casedata))
casedata = pd.read_pickle('casedata.pkl')

In [15]:
columns1=['CaseId','Date','Vector','Year','Month']


i=0
docvector= pd.DataFrame(columns=columns1)
for caseid in docLabels:
    
    date=casedata.loc[caseid]["Date"]
    m=casedata.loc[caseid]["Month"]
    try:
        m1=int(m)
        y=int(casedata.loc[caseid]["Year"])
    except:
        print(casedata.loc[caseid])
        continue
        
        
       
    
    
    
    docvec = d2v_model.docvecs[caseid]
    docvector = docvector.append({'CaseId':caseid,'Date':date,'Vector':docvec,'Month':m1,'Year':y},ignore_index=True)
print(len(docvector))
docvector.head(10)

            Year     Date Month                               filename Day
CaseId                                                                    
X1ESBE0003  2011  1/20/11     1  X1ESBE0003_contentMajOp_TORRUELLA.txt  20
X1ESBE0003  2011  1/20/11     1      X1ESBE0003_contentMajOp_LIPEZ.txt  20
          Year      Date Month                          filename Day
CaseId                                                              
X1EF18GN  2012  10/23/12    10  X1EF18GN_contentMajOp_PLAGER.txt  23
X1EF18GN  2012  10/23/12    10  X1EF18GN_contentMajOp_NEWMAN.txt  23
            Year      Date Month                           filename Day
CaseId                                                                 
X1J28GS003  2012  12/21/12    12   X1J28GS003_contentMajOp_ROTH.txt  21
X1J28GS003  2012  12/21/12    12  X1J28GS003_contentMajOp_SMITH.txt  21
            Year      Date Month                           filename Day
CaseId                                                          

,CaseId,Date,Vector,Year,Month
0,X1EUV2Q003,2/9/11,"[-70.229, -39.9497, -114.603, -55.2935, 42.171...",2011,2
1,X1F5E12003,3/22/11,"[20.8616, -62.7381, -34.9262, 15.7718, 49.0267...",2011,3
2,X1EVBPM003,2/14/11,"[-20.0328, -24.5839, -95.1057, -75.5293, 20.78...",2011,2
3,X1G417C003,8/19/11,"[-11.3081, -99.5218, -54.6434, 77.7286, 95.301...",2011,8
4,X1F326S003,3/9/11,"[97.1497, -32.4867, 53.0597, 62.5793, -74.615,...",2011,3
5,X1EVNDK003,2/17/11,"[29.3697, -45.7484, 7.98869, 70.2024, 40.8673,...",2011,2
6,X1EUGG2003,2/4/11,"[-28.7306, -70.5179, 15.0418, 29.471, 27.9751,...",2011,2
7,X1EPPSE003,1/4/11,"[-152.679, -46.9773, -129.137, 102.765, 18.625...",2011,1
8,X1GLH6G003,10/31/11,"[36.1238, -68.1266, -8.55172, 72.4784, -18.601...",2011,10
9,X1F7JTM003,4/4/11,"[5.26688, -39.9097, -62.5562, 23.5115, -8.4668...",2011,4


In [16]:
#finding similar docs
sims = d2v_model.docvecs.most_similar('XVITNB80000N')
print((sims))
print(len(docvector))

[('X1I1GKG003', 0.8351005911827087), ('X1F8IVM003', 0.8231872916221619), ('X1ISME8003', 0.800939679145813), ('X1EVADQ003', 0.7985252737998962), ('X1FT60S003', 0.7940493822097778), ('X1IQ5BU003', 0.7881262898445129), ('X1HB8VC003', 0.7830334305763245), ('X1I2O08003', 0.77363520860672), ('X1G295Q003', 0.7721319794654846), ('X1FIUV0003', 0.7702466249465942)]
1655


In [17]:
docvector.to_pickle("docvector.pkl")

In [18]:

with open('docvector.pkl', 'rb') as f:
    docvec = pickle.load(f)
docvec.head(3)

,CaseId,Date,Vector,Year,Month
0,X1EUV2Q003,2/9/11,"[-70.229, -39.9497, -114.603, -55.2935, 42.171...",2011,2
1,X1F5E12003,3/22/11,"[20.8616, -62.7381, -34.9262, 15.7718, 49.0267...",2011,3
2,X1EVBPM003,2/14/11,"[-20.0328, -24.5839, -95.1057, -75.5293, 20.78...",2011,2


In [21]:
train=docvec[docvec["Year"]!=2013]
print(len(train))      
train.head(3)

1178


,CaseId,Date,Vector,Year,Month
0,X1EUV2Q003,2/9/11,"[-70.229, -39.9497, -114.603, -55.2935, 42.171...",2011,2
1,X1F5E12003,3/22/11,"[20.8616, -62.7381, -34.9262, 15.7718, 49.0267...",2011,3
2,X1EVBPM003,2/14/11,"[-20.0328, -24.5839, -95.1057, -75.5293, 20.78...",2011,2


In [22]:
test=docvec[docvec["Year"]==2013]
print(len(test)) 
test.head(3)

477


,CaseId,Date,Vector,Year,Month
1178,X1HT4F0N,5/21/13,"[-85.6824, -87.9674, 7.54578, 89.8588, 94.791,...",2013,5
1179,X1JB7OQ003,2/28/13,"[-119.401, -51.1509, -99.7955, -31.6389, -47.4...",2013,2
1180,X1J8UPE003,2/7/13,"[-15.9718, -129.339, -133.165, 66.1679, 118.14...",2013,2


In [23]:
train.to_pickle("traindocvector.pkl")
test.to_pickle("testdocvector.pkl")

In [24]:
tt = pd.read_pickle('traindocvector.pkl')

In [25]:
tt


,CaseId,Date,Vector,Year,Month
0,X1EUV2Q003,2/9/11,"[-70.229, -39.9497, -114.603, -55.2935, 42.171...",2011,2
1,X1F5E12003,3/22/11,"[20.8616, -62.7381, -34.9262, 15.7718, 49.0267...",2011,3
2,X1EVBPM003,2/14/11,"[-20.0328, -24.5839, -95.1057, -75.5293, 20.78...",2011,2
3,X1G417C003,8/19/11,"[-11.3081, -99.5218, -54.6434, 77.7286, 95.301...",2011,8
4,X1F326S003,3/9/11,"[97.1497, -32.4867, 53.0597, 62.5793, -74.615,...",2011,3
5,X1EVNDK003,2/17/11,"[29.3697, -45.7484, 7.98869, 70.2024, 40.8673,...",2011,2
6,X1EUGG2003,2/4/11,"[-28.7306, -70.5179, 15.0418, 29.471, 27.9751,...",2011,2
7,X1EPPSE003,1/4/11,"[-152.679, -46.9773, -129.137, 102.765, 18.625...",2011,1
8,X1GLH6G003,10/31/11,"[36.1238, -68.1266, -8.55172, 72.4784, -18.601...",2011,10
9,X1F7JTM003,4/4/11,"[5.26688, -39.9097, -62.5562, 23.5115, -8.4668...",2011,4


In [26]:
tt1 = pd.read_pickle("testdocvector.pkl")
tt1

,CaseId,Date,Vector,Year,Month
1178,X1HT4F0N,5/21/13,"[-85.6824, -87.9674, 7.54578, 89.8588, 94.791,...",2013,5
1179,X1JB7OQ003,2/28/13,"[-119.401, -51.1509, -99.7955, -31.6389, -47.4...",2013,2
1180,X1J8UPE003,2/7/13,"[-15.9718, -129.339, -133.165, 66.1679, 118.14...",2013,2
1181,X1JEEKK003,3/25/13,"[-17.2584, -36.0479, -36.9193, 55.0872, 47.008...",2013,3
1182,X1JSOJI003,7/12/13,"[35.2335, -20.4723, 60.7124, 28.0635, -48.7737...",2013,7
1183,X1JKU4G003,5/13/13,"[3.35522, -64.7331, -134.811, 44.3746, 33.7792...",2013,5
1184,X1GERHON,3/28/13,"[25.2665, -123.05, -59.2467, 29.4108, 83.1448,...",2013,3
1185,X1JB2P6003,2/27/13,"[-48.5661, -45.8015, -180.899, -110.735, 85.30...",2013,2
1186,X1JTL1Q003,7/18/13,"[-9.24227, -74.9902, -60.1695, 81.6972, 35.062...",2013,7
1187,X1F0UMON,1/10/13,"[47.3564, 0.0856834, -26.3397, -45.9103, 33.89...",2013,1
